# Preliminary Notebook

## 1. Import and Install Dependencies

In [ ]:
%pip install tensorflow-macos opencv-python mediapipe-silicon sklearn matplotlib
#!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib # original code line from tutorial (he used a Windows system)

Note: you may need to restart the kernel to use updated packages.


In [2]:
# general 
import numpy as np
import pandas as pd
import os # easier file path handling

# for device camera feed
import cv2 # opencv
from matplotlib import pyplot as plt # imshow for easy visualization
import time # to insert breaks / "sleep" in between frames
import mediapipe as mp # for accessing and reading from device camera

# for data pre-processing
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# for model evaluation
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

objc[6500]: Class CaptureDelegate is implemented in both /Users/jin-holee/neuefische/Capstone_Project_SignMeUp/.venv/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x12d6b25a0) and /Users/jin-holee/neuefische/Capstone_Project_SignMeUp/.venv/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x129554860). One of the two will be used. Which one is undefined.
objc[6500]: Class CVWindow is implemented in both /Users/jin-holee/neuefische/Capstone_Project_SignMeUp/.venv/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x12d6b25f0) and /Users/jin-holee/neuefische/Capstone_Project_SignMeUp/.venv/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x129250a68). One of the two will be used. Which one is undefined.
objc[6500]: Class CVView is implemented in both /Users/jin-holee/neuefische/Capstone_Project_SignMeUp/.venv/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x12d6b2618) and /Users/jin-holee/neuefische/Capstone_Project_SignMeUp/.venv/lib/pytho

## 6. Preprocess Data and Create Labels and Features

### Some global stuff (edit to your needs)

In [4]:
# running modes 'test' or 'kaggle'. Default is using own generated data.
MODE = 'kaggle'

# path for input data and globals
if MODE == 'test': 
    DATA_PATH = os.path.join('MP_Data_test')
elif MODE == 'kaggle': 
    DATA_PATH = os.path.join('../../asl-signs') # path to our Kaggle
else: 
    DATA_PATH = os.path.join('MP_Data')

actions = np.array(['hello', 'thanks', 'iloveyou']) # actions to detect (get later from our .json file)
no_sequences = 30 # 30 videos per sequence / word / sign (get it later from input file)
sequence_length = 30 # each video with 30 frames (get it later from input file)

In [5]:
# create label map (dict, later our .json file)
label_map = {label:num for num, label in enumerate(actions)}
label_map = 

In [6]:
sequences, labels = [], [] # sequences will be x data, labels will be y data
# loop over all actions (words)
for action in actions: 
    # loop over all sequences (videos)
    for sequence in range(no_sequences): 
        window = [] # represents all frames of particular sequence (video)
        # loop through each frame
        for frame_num in range(sequence_length): 
            # load up current frame (frame_num)
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res) # append to one video
        sequences.append(window) # append all videos to sequence (for a word)
        labels.append(label_map[action])

## Here, we could feed in our Kaggle data

In [7]:
X = np.array(sequences) # shape: (90, 30, 1662)

In [8]:
y = to_categorical(labels).astype(int) # one-hot encoded labels (words)
# shape: (90, 3)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

## 7. Build and Train LSTM Neural Network

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [11]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [12]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [13]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# categorical_crossentropy must be used for multiclass classification model! 

In [14]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])
# advantage of using mediapipe holistic model is you don't need additional data generator to build up a pipeline of data. Training data fits all into memory.

Epoch 1/2000


2023-04-05 16:42:07.807294: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


3/3 [==============================] - 2s 35ms/step - loss: 5.8393 - categorical_accuracy: 0.3412
Epoch 2/2000
3/3 [==============================] - 0s 38ms/step - loss: 7.2594 - categorical_accuracy: 0.3529
Epoch 3/2000
3/3 [==============================] - 0s 46ms/step - loss: 1.4725 - categorical_accuracy: 0.2471
Epoch 4/2000
3/3 [==============================] - 0s 61ms/step - loss: 1.5703 - categorical_accuracy: 0.3765
Epoch 5/2000
3/3 [==============================] - 0s 39ms/step - loss: 1.1826 - categorical_accuracy: 0.3647
Epoch 6/2000
3/3 [==============================] - 0s 38ms/step - loss: 1.7082 - categorical_accuracy: 0.4000
Epoch 7/2000
3/3 [==============================] - 0s 37ms/step - loss: 1.6134 - categorical_accuracy: 0.2941
Epoch 8/2000
3/3 [==============================] - 0s 39ms/step - loss: 2.3487 - categorical_accuracy: 0.2588
Epoch 9/2000
3/3 [==============================] - 0s 60ms/step - loss: 1.2106 - categorical_accuracy: 0.3647
Epoch 10/2000


In [15]:
# %load_ext tensorboard

In [16]:
# to run it in the notebook: 
# %tensorboard --logdir=./Logs/train --port=8008

# to run it in terminal: python3 -m tensorboard --logdir=./Logs/train --port=8008
# then copy+paste this into your internet browser: localhost:8008

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

## Evaluation using Confusion Matrix and Accuracy

Some single example predictions :D

In [19]:
actions[np.argmax(res[0])] # prediction

'hello'

In [20]:
actions[np.argmax(y_test[0])] # actual

'hello'

In [ ]:
yhat = model.predict(X_test)

2023-04-05 22:18:54.815757: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 272ms/step


In [ ]:
# convert one-hot encoded categories to labels, e.g. 0, 1 and 2
# instead of [1,0,0], [0,1,0], [0,0,1]
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[0, 4],
        [0, 1]],

       [[1, 0],
        [4, 0]]])

In [ ]:
accuracy_score(ytrue, yhat)

0.2

## Save Model Weights

In [21]:
model.save('first_model_whoop_whoop.h5')